In [2]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.stats import t

In [3]:
# load in data here
df1 = pd.read_csv("imnemato-youtube[1080p]-1x-vpn-mac-clean-20201031a.csv")
df2 = pd.read_csv("vpn_browsing1.csv")

df3 = pd.read_csv("pgaddiso-youtube-1x-720p-vpn-linux-clean-20201102-1.csv")
df4 = pd.read_csv("vpn_browsing2.csv")


df5 = windows_video_vpn = pd.read_csv("stdoan-youtube[720p60]-1x-vpn-windows-clean-20201102.csv")
df6 = pd.read_csv("vpn_browsing3.csv")

# make sure that dataframes are in list
list_of_frames = [df1, df2, df3,df4,df5,df6]
#add vpn dataframes to next list
video_frames = [df1,df3,df5]
browsing_frames = [df2,df4,df6]

In [4]:
# data cleaning
# helper method to sum all package lengths for each entry in a column
def package(input1):
    sum1 = 0
    for x in input1:
        if x  != "":
            sum1 += int(x)
    return sum1

# adds total pacakge length to all dataframes
def packet_length(frames):
    for j in frames:
        j["sum of sizes"] = pd.Series(j.packet_sizes.apply(lambda x: x.split(";"))).apply(lambda x: package(x))

#label the data
def label():
    for x in video_frames:
            x["Label"] = 1
    for j in browsing_frames:
            j["Label"] = 0


In [5]:
# labes data 
label()

# total packet length
packet_length(list_of_frames)






In [6]:
def ml_model_data(frames):
    # global_varaiables for data storage
    X_train_df =  None
    X_test_df  = None
    y_train_df = None
    y_test_df = None

    for f in frames:
        #isolate features in frames
        X = f[["sum of sizes", "1->2Bytes","2->1Bytes"]]
        
        #isolate label
        y = f[["Label"]]
        
        #split data between training and test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 
       
        #gatther data from all dataframes
        X_train_df = pd.concat([X_train_df, X_train])
        X_test_df = pd.concat([X_test_df, X_test])
        y_train_df = pd.concat([y_train_df, y_train])
        y_test_df = pd.concat([y_test_df, y_test])
        
        
        
    return [X_train_df,X_test_df,y_train_df,y_test_df]
        
    

In [7]:
All_data = ml_model_data(list_of_frames)

In [8]:
def making_predictions(ml_model_data):
    clf = LogisticRegression()
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1])
    
    return predictions
    

In [9]:
predictions = making_predictions(All_data)

C:\Users\dania\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\dania\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# clf = LogisticRegression()
# clf.fit(All_data[0], All_data[1])
# predictions = clf.predict(ml_model_data[2])
# predictions
mse = mean_squared_error(All_data[3]["Label"], predictions)

In [11]:
def confidence_interval(prediction, confidence):
    min_value = min(prediction)
    max_value = max(prediction)
    
    average_list = []
    
    for x in range(1000):
        samples = np.random.choice(prediction, len(prediction))
        average_list.append(np.mean(samples))
    
    size = len(average_list)
    average = np.mean(average_list)
    st_error = stats.sem(average_list)
    interval = st_error * t.ppf((1 + confidence)/2,size-1)
    low = round(average- interval)
    up = round(average +interval)
    return (low, up)

In [12]:
interval =confidence_interval(predictions,0.95)

In [13]:
print("The prdicitons from my ML model are", predictions)

The prdicitons from my ML model are [1 1 1 ... 1 1 1]


In [14]:
print("The mean square error of my model is" , mse)

The mean square error of my model is 0.7250932338838573


In [15]:
print("The confidence intervale of my model is ",interval )

The confidence intervale of my model is  (1.0, 1.0)
